In [2]:
# load modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip

In [95]:
# load files
# filename = "data/nairobi_airport.gz"
filename = "data/kisumu_weather.gz"
df = pd.read_csv(filename,sep=";",skiprows=6,
                 encoding='UTF-8', low_memory=False)


In [96]:
df.head(5)

,Local time in Kisumu (airport),T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
28.12.2020 15:00,27.7,NaN,NaN,NaN,44.0,Wind blowing from the south-southwest,4.0,NaN,NaN,"10% or less, but not 0",...,"Cirrus fibratus, sometimes uncinus, not progre...",30.0,14.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28.12.2020 12:00,26.3,NaN,NaN,NaN,47.0,Wind blowing from the west-southwest,3.0,NaN,NaN,"10% or less, but not 0",...,"Cirrus fibratus, sometimes uncinus, not progre...",30.0,14.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28.12.2020 09:00,23.8,NaN,NaN,NaN,58.0,Wind blowing from the east,2.0,NaN,NaN,"10% or less, but not 0",...,"Cirrus fibratus, sometimes uncinus, not progre...",30.0,15.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28.12.2020 06:00,17.6,NaN,NaN,NaN,82.0,Wind blowing from the east-northeast,1.0,NaN,NaN,20–30%.,...,"Cirrus fibratus, sometimes uncinus, not progre...",30.0,14.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28.12.2020 03:00,18.6,NaN,NaN,NaN,94.0,"Calm, no wind",0.0,NaN,NaN,40%.,...,"Cirrus fibratus, sometimes uncinus, not progre...",30.0,17.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
df1 = df.reset_index()
df1.head()

,index,Local time in Kisumu (airport),T,Po,P,Pa,U,DD,Ff,ff10,...,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
0,28.12.2020 15:00,27.7,NaN,NaN,NaN,44.0,Wind blowing from the south-southwest,4.0,NaN,NaN,...,"Cirrus fibratus, sometimes uncinus, not progre...",30.0,14.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28.12.2020 12:00,26.3,NaN,NaN,NaN,47.0,Wind blowing from the west-southwest,3.0,NaN,NaN,...,"Cirrus fibratus, sometimes uncinus, not progre...",30.0,14.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28.12.2020 09:00,23.8,NaN,NaN,NaN,58.0,Wind blowing from the east,2.0,NaN,NaN,...,"Cirrus fibratus, sometimes uncinus, not progre...",30.0,15.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28.12.2020 06:00,17.6,NaN,NaN,NaN,82.0,Wind blowing from the east-northeast,1.0,NaN,NaN,...,"Cirrus fibratus, sometimes uncinus, not progre...",30.0,14.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28.12.2020 03:00,18.6,NaN,NaN,NaN,94.0,"Calm, no wind",0.0,NaN,NaN,...,"Cirrus fibratus, sometimes uncinus, not progre...",30.0,17.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
df.tail(2)

,Local time in Kisumu (airport),T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
29.01.2009 03:00,20.3,NaN,NaN,NaN,57.0,Wind blowing from the east,1.0,NaN,NaN,"10% or less, but not 0",...,"Cirrus fibratus, sometimes uncinus, not progre...",25.0,11.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06.04.2005 09:00,21.6,NaN,NaN,NaN,83.0,"Calm, no wind",0.0,NaN,NaN,70 – 80%.,...,"Cirrocumulus alone, or Cirrocumulus accompanie...",35.0,18.6,6.0,24.0,NaN,NaN,NaN,NaN,NaN


In [99]:
df.shape

(22554, 29)

In [60]:
[x for x in df.columns]

['Local time in Nairobi (airport)',
 'T',
 'Po',
 'P',
 'Pa',
 'U',
 'DD',
 'Ff',
 'ff10',
 'ff3',
 'N',
 'WW',
 'W1',
 'W2',
 'Tn',
 'Tx',
 'Cl',
 'Nh',
 'H',
 'Cm',
 'Ch',
 'VV',
 'Td',
 'RRR',
 'tR',
 'E',
 'Tg',
 "E'",
 'sss']